# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-multiple-stock-trading-7b00763b7530
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- ASSUMES USING DOCKER, see readme for instructions

<a id='1.1'></a>
## 2.1. Add FinRL to your path. You can of course install it as a pipy package, but this is for development purposes.


In [1]:
# import sys

# sys.path.append("..")

In [2]:
# import pandas as pd
# print(pd.__version__)


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
# from finrl.finrl_meta.env_stock_trading.env_stocktrading_cashpenalty import StockTradingEnvCashpenalty\
# from finrl.finrl_meta.env_stock_trading.env_stocktrading_cashpenalty2 import StockTradingEnvCashpenalty
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.plot import backtest_plot_tdx

from lutils.stock.env.env_stocktrading_cashpenalty import StockTradingEnvCashpenalty

# import sys
# sys.path.append("../FinRL-Library")

d:\code\python\pyfolio\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


Logging to /home/l/code/python/log


In [4]:
from lutils.stock import LTdxHq

<a id='1.4'></a>
## 2.4. Create Folders

In [5]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [6]:
# from config.py start_date is a string
# config.START_DATE

In [7]:
# from config.py end_date is a string
# config.END_DATE

In [8]:
# # print(config.DOW_30_TICKER)
# stock_codes = ['600603', '600583', '600582', '600565', '600558', 
#                '600551', '600509', '600503', '600488', '600469', 
#                '600439', '600433', '600428', ]#'600425', '600387', ]
# #                '600383', '600382', '600369', '600339', '600337', ]
# #                '600335', '600326', '600320', '600312', '002666', 
# #                '002654', '002641', '002628', '002627', '002620', 
# #                '002608', '002593', '002586', '002582', '002566', '002560',]

In [9]:
stock_codes = ['000921', '600365', '000609', '601868', '002372', '001965']

In [10]:
# df = YahooDownloader(start_date = '2009-01-01',
#                      end_date = '2021-12-13',
#                      ticker_list = config.DOW_30_TICKER).fetch_data()

In [11]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2010-01-01')
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s %s -----------' % (code, df.index.min()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 000921 2010-01-04 00:00:00 -----------
----------- over 600365 2010-01-04 00:00:00 -----------
----------- over 000609 2010-01-04 00:00:00 -----------
----------- over 601868 2021-09-28 00:00:00 -----------
----------- over 002372 2010-03-18 00:00:00 -----------
----------- over 001965 2017-12-25 00:00:00 -----------


In [12]:
# df1 = dfs[0].reindex(indexs)
# df2 = dfs[1].reindex(indexs)

In [13]:
# df1.ffill()['2010-01-02':]

In [14]:
# df3 = df2.ffill()

In [15]:
# dfs[1][dfs[-3].close.isna()]

In [16]:
# def clean_data(data):
#     df = data.copy()
#     df = df.sort_values(["date", "tic"], ignore_index=True)
#     df.index = df.date.factorize()[0]
#     merged_closes = df.pivot_table(index="date", columns="tic", values="close")
# #     print(merged_closes)
# #     merged_closes = merged_closes.dropna(axis=1)
# #     print(merged_closes)
#     tics = merged_closes.columns
# #     print(tics)
#     df = df[df.tic.isin(tics)]
# #     print(df)
#     return df

In [17]:
# print(clean_data(df))

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [18]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


In [19]:
processed['log_volume'] = np.log(processed.volume*processed.close)
processed['change'] = (processed.close-processed.open)/processed.close
processed['daily_variance'] = (processed.high-processed.low)/processed.close
processed.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,log_volume,change,daily_variance
0,11.93,11.78,12.08,11.77,2890059.0,34479184.0,000609,2010-01-04,0,0.000000,11.990269,11.339731,0.0,-66.666667,100.0,11.780,11.780,17.343191,-0.012733,0.026316
2930,7.25,7.31,7.43,7.17,6787037.0,58770860.0,000921,2010-01-04,0,0.000000,11.990269,11.339731,0.0,-66.666667,100.0,7.310,7.310,17.719768,0.008208,0.035568
5860,5.94,5.84,5.95,5.76,7031153.0,82264104.0,600365,2010-01-04,0,0.000000,11.990269,11.339731,0.0,-66.666667,100.0,5.840,5.840,17.530592,-0.017123,0.032534
1,11.77,11.55,11.90,11.48,4026759.0,46896480.0,000609,2010-01-05,1,-0.005160,11.990269,11.339731,0.0,-66.666667,100.0,11.665,11.665,17.655158,-0.019048,0.036364
2931,7.32,7.59,7.62,7.21,8105622.0,70827248.0,000921,2010-01-05,1,0.006282,7.845980,7.054020,100.0,66.666667,100.0,7.450,7.450,17.934900,0.035573,0.054018


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2016-01-01
## Trade data split: 2016-01-01 to 2021-01-01

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2016-01 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [20]:
train = data_split(processed, '2016-01-01','2021-01-01') # '2009-01-01','2016-01-01'
trade = data_split(processed, '2021-01-01','2021-12-20') # '2016-01-01','2021-01-01'
print(len(train))
print(len(trade))

3654
699


In [21]:
print(StockTradingEnvCashpenalty.__doc__)


    A stock trading environment for OpenAI gym
    This environment penalizes the model for not maintaining a reserve of cash.
    This enables the model to manage cash reserves in addition to performing trading procedures.
    Reward at any step is given as follows
        r_i = (sum(cash, asset_value) - initial_cash - max(0, sum(cash, asset_value)*cash_penalty_proportion-cash))/(days_elapsed)
        This reward function takes into account a liquidity requirement, as well as long-term accrued rewards.
    Parameters:
        df (pandas.DataFrame): Dataframe containing data
        buy_cost_pct (float): cost for buying shares
        sell_cost_pct (float): cost for selling shares
        hmax (int, array): maximum cash to be traded in each trade per asset. If an array is provided, then each index correspond to each asset
        discrete_actions (bool): option to choose whether perform dicretization on actions space or not
        shares_increment (int): multiples number of shares ca

#### state space
The state space of the observation is as follows 

`start_cash, <owned_shares_of_n_assets>, <<indicator_i_for_asset_j> for j in assets>`

indicators are any daily measurement you can achieve. Common ones are 'volume', 'open' 'close' 'high', 'low'.
However, you can add these as needed, 
The feature engineer adds indicators, and you can add your own as well. 


In [23]:
processed[processed['cci_30'] == -np.inf]

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,log_volume,change,daily_variance


In [24]:
processed.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,log_volume,change,daily_variance
0,11.93,11.78,12.08,11.77,2890059.0,34479184.0,000609,2010-01-04,0,0.000000,11.990269,11.339731,0.0,-66.666667,100.0,11.780,11.780,17.343191,-0.012733,0.026316
2930,7.25,7.31,7.43,7.17,6787037.0,58770860.0,000921,2010-01-04,0,0.000000,11.990269,11.339731,0.0,-66.666667,100.0,7.310,7.310,17.719768,0.008208,0.035568
5860,5.94,5.84,5.95,5.76,7031153.0,82264104.0,600365,2010-01-04,0,0.000000,11.990269,11.339731,0.0,-66.666667,100.0,5.840,5.840,17.530592,-0.017123,0.032534
1,11.77,11.55,11.90,11.48,4026759.0,46896480.0,000609,2010-01-05,1,-0.005160,11.990269,11.339731,0.0,-66.666667,100.0,11.665,11.665,17.655158,-0.019048,0.036364
2931,7.32,7.59,7.62,7.21,8105622.0,70827248.0,000921,2010-01-05,1,0.006282,7.845980,7.054020,100.0,66.666667,100.0,7.450,7.450,17.934900,0.035573,0.054018


In [25]:
information_cols = ['daily_variance', 'change', 'log_volume', 'close','day', 
                    'macd', 'rsi_30', 'boll_ub', 'dx_30'] # cci_30

e_train_gym = StockTradingEnvCashpenalty(
    df = train,
    initial_amount = 50e4, # 1e6, 
    hmax = 2000, 
    turbulence_threshold = None, 
    currency='￥',
    buy_cost_pct=3e-3,
    sell_cost_pct=3e-3,
    cash_penalty_proportion=0.2,
    cache_indicator_data=True,
    daily_information_cols = information_cols,
    print_verbosity = 500, 
    random_start = True,
    ) # patient = True

e_trade_gym = StockTradingEnvCashpenalty(
    df = trade,
    initial_amount = 50e4, # 1e6, 
    hmax = 2000, 
    turbulence_threshold = None, 
    currency='￥',
    buy_cost_pct=3e-3,
    sell_cost_pct=3e-3,
    cash_penalty_proportion=0.2,
    cache_indicator_data=True,
    daily_information_cols = information_cols, 
    print_verbosity = 500, 
    random_start = False,
    ) # patient = True

caching data
data cached!
caching data
data cached!


## Environment for Training
There are two available environments. The multiprocessing and the single processing env. 
Some models won't work with multiprocessing. 

```python
# single processing
env_train, _ = e_train_gym.get_sb_env()


#multiprocessing
env_train, _ = e_train_gym.get_multiproc_env(n = <n_cores>)
```


In [26]:
# for this example, let's do multiprocessing with n_cores-2
import multiprocessing

# n_cores = multiprocessing.cpu_count() - 2
n_cores = 3
# print(f"using {n_cores} cores")

#this is our training env. It allows multiprocessing
# env_train, _ = e_train_gym.get_multiproc_env(n = n_cores)
env_train, _ = e_train_gym.get_sb_env()

#this is our observation environment. It allows full diagnostics
env_trade, _ = e_trade_gym.get_sb_env()

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [27]:
agent = DRLAgent(env = env_train)

### Model PPO


In [28]:
from torch.nn import Softsign, ReLU

ppo_params ={'n_steps': 256, 
             'ent_coef': 0.0, 
             'learning_rate': 0.0000002, 
             'batch_size': 1024, 
             'gamma': 0.99}

policy_kwargs = {
#     "activation_fn": ReLU, 
    "net_arch": [1024 for _ in range(10)], 
#     "squash_output": True
}

model = agent.get_model('ppo', # ppo, a2c
                        model_kwargs = ppo_params, 
                        policy_kwargs = policy_kwargs, 
                        tensorboard_log = '/home/l/code/python/log/run',
                        verbose = 0)

# model = model.load("scaling_reward.model", env = env_train)

{'n_steps': 256, 'ent_coef': 0.0, 'learning_rate': 2e-07, 'batch_size': 1024, 'gamma': 0.99}


d:\code\python\stable-baselines3\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 1024, but because the `RolloutBuffer` is of size `n_steps * n_envs = 256`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 256
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=256 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [29]:
model.learn(total_timesteps = 200000, # 5000000
            eval_env = env_trade, 
            eval_freq = 500, 
            log_interval = 1, 
            tb_log_name = 'env_cashpenalty_highlr', 
            n_eval_episodes = 2)

d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1| 499|update         |￥461,611       |￥483,751       |-0.00653% |-3.24975% |95.42%    
EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION


d:\code\python\stable-baselines3\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


   1| 232|Last Date      |￥497,319       |￥499,935       |-0.00006% |-0.01291% |99.48%    
   2| 232|Last Date      |￥497,319       |￥499,935       |-0.00006% |-0.01291% |99.48%    
Eval num_timesteps=500, episode_reward=-0.00 +/- 0.00
Episode length: 233.00 +/- 0.00
New best mean reward!
   1| 814|Last Date      |￥395,794       |￥464,586       |-0.00870% |-7.08272% |85.19%    
   4| 232|Last Date      |￥496,532       |￥499,917       |-0.00007% |-0.01669% |99.32%    
   5| 232|Last Date      |￥496,532       |￥499,917       |-0.00007% |-0.01669% |99.32%    
Eval num_timesteps=1000, episode_reward=-0.00 +/- 0.00
Episode length: 233.00 +/- 0.00
   2| 499|update         |￥439,420       |￥481,214       |-0.00754% |-3.75725% |91.31%    
   7| 232|Last Date      |￥495,422       |￥499,890       |-0.00009% |-0.02204% |99.11%    
   8| 232|Last Date      |￥495,422       |￥499,890       |-0.00009% |-0.02204% |99.11%    
Eval num_timesteps=1500, episode_reward=-0.00 +/- 0.00
Episode length: 233.00

  12|1070|Last Date      |￥429,096       |￥485,410       |-0.00273% |-2.91809% |88.40%    
  64| 232|Last Date      |￥495,157       |￥501,235       |0.00106%  |0.24704%  |98.79%    
  65| 232|Last Date      |￥495,157       |￥501,235       |0.00106%  |0.24704%  |98.79%    
Eval num_timesteps=11000, episode_reward=0.00 +/- 0.00
Episode length: 233.00 +/- 0.00
  13| 499|update         |￥394,236       |￥479,474       |-0.00824% |-4.10512% |82.22%    
  67| 232|Last Date      |￥495,049       |￥501,266       |0.00109%  |0.25317%  |98.76%    
  68| 232|Last Date      |￥495,049       |￥501,266       |0.00109%  |0.25317%  |98.76%    
Eval num_timesteps=11500, episode_reward=0.00 +/- 0.00
Episode length: 233.00 +/- 0.00
  13| 999|update         |￥305,648       |￥474,762       |-0.00506% |-5.04757% |64.38%    
  13|1121|Last Date      |￥303,335       |￥443,940       |-0.01000% |-11.21208%|68.33%    
  70| 232|Last Date      |￥494,079       |￥501,528       |0.00132%  |0.30566%  |98.51%    
  71| 2

 125| 232|Last Date      |￥486,045       |￥503,700       |0.00319%  |0.74004%  |96.49%    
Eval num_timesteps=21000, episode_reward=0.01 +/- 0.00
Episode length: 233.00 +/- 0.00
  24| 999|update         |￥372,334       |￥468,247       |-0.00636% |-6.35059% |79.52%    
 127| 232|Last Date      |￥486,168       |￥503,650       |0.00315%  |0.72990%  |96.53%    
 128| 232|Last Date      |￥486,168       |￥503,650       |0.00315%  |0.72990%  |96.53%    
Eval num_timesteps=21500, episode_reward=0.01 +/- 0.00
Episode length: 233.00 +/- 0.00
  24|1046|Last Date      |￥361,979       |￥479,937       |-0.00384% |-4.01259% |75.42%    
 130| 232|Last Date      |￥485,929       |￥503,825       |0.00330%  |0.76501%  |96.45%    
 131| 232|Last Date      |￥485,929       |￥503,825       |0.00330%  |0.76501%  |96.45%    
Eval num_timesteps=22000, episode_reward=0.01 +/- 0.00
Episode length: 233.00 +/- 0.00
  25| 499|update         |￥445,251       |￥472,144       |-0.01119% |-5.57124% |94.30%    
 133| 232|L

KeyboardInterrupt: 

In [28]:
# model.save('different1_24.model')

In [30]:
obs = env_trade.reset()

done, state = False, None
# while not done:
for _ in range(50):
    action, state = model.predict(obs, state=state, deterministic=True)
    obs, reward, done, _info = env_trade.step(action)

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [31]:
trade.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,log_volume,change,daily_variance
0,4.64,4.74,4.84,4.60,93897448.0,445941472.0,600428,2021-01-04,0,0.139979,4.783990,3.585010,59.778697,105.466593,43.189911,4.297667,3.944000,19.913751,0.021097,0.050633
0,4.71,4.80,4.92,4.70,83298224.0,404595648.0,600433,2021-01-04,0,0.149423,4.668935,3.868065,65.988517,186.383552,49.396656,4.191333,3.827000,19.806554,0.018750,0.045833
0,2.47,2.47,2.50,2.43,10972837.0,26985688.0,600439,2021-01-04,0,-0.122469,2.993202,2.413798,32.774391,-145.161290,68.589322,2.806000,2.908167,17.115152,0.000000,0.028340
0,5.22,5.32,5.40,5.20,14183933.0,76075184.0,600469,2021-01-04,0,0.034947,5.265100,4.719900,57.971339,129.209622,47.175939,5.054333,4.981500,18.139094,0.018797,0.037594
0,4.93,4.92,4.95,4.90,7965500.0,39298684.0,600488,2021-01-04,0,-0.020858,5.263111,4.775889,47.171897,-96.316131,0.318032,5.050000,5.039667,17.483939,-0.002033,0.010163


In [32]:
e_trade_gym.hmax = 5000

In [33]:
print(len(e_trade_gym.dates))

233


In [34]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=model, environment=e_trade_gym)

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1| 232|Last Date      |￥311,178       |￥490,365       |-0.00831% |-1.92703% |63.46%    
hit end!


In [35]:
df_actions.head()

,date,actions,transactions
0,2021-01-05,"[0.32822755, -0.41228452, -0.5542212, 1.0, -1....","[346.23158128955697, -0.0, -0.0, 939.849624060..."
1,2021-01-06,"[-0.90147626, 1.0, -0.4065298, 1.0, -0.7086854...","[-346.23158128955697, 1039.5010395010395, -0.0..."
2,2021-01-07,"[1.0, 0.21601456, 1.0, -0.92499864, 0.13175616...","[1086.9565217391305, 214.3001738048735, 2145.9..."
3,2021-01-08,"[-1.0, 0.13374579, 1.0, -0.5376402, -0.3108284...","[-1086.9565217391305, 139.60938264532228, 2222..."
4,2021-01-11,"[-0.7230543, -0.2935453, -0.59456563, 0.528749...","[-0.0, -308.9950657894737, -1198.7210181451612..."


In [36]:
df_account_value.shape

(232, 5)

In [37]:
df_account_value.head(50)

,cash,asset_value,total_assets,reward,date
0,500000.000000,0.000000,500000.000000,0.000000,2021-01-05
1,481549.476096,18693.033629,500242.509725,0.000485,2021-01-06
2,475304.300641,25078.964579,500383.265220,0.000383,2021-01-07
3,455925.521043,42618.180811,498543.701854,-0.000971,2021-01-08
4,457595.254875,41659.985260,499255.240135,-0.000372,2021-01-11
5,460219.915032,38302.390712,498522.305744,-0.000591,2021-01-12
6,459138.177791,40804.681755,499942.859546,-0.000019,2021-01-13
7,466973.465477,33293.339753,500266.805230,0.000076,2021-01-14
8,467818.821158,31673.581570,499492.402728,-0.000127,2021-01-15
9,448403.141739,51955.337943,500358.479682,0.000080,2021-01-18


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [38]:
print("==============Get Backtest Results===========")
perf_stats_all = backtest_stats(account_value=df_account_value, value_col_name = 'total_assets')

==============Get Backtest Results===========
Annual return         -0.020914
Cumulative returns    -0.019270
Annual volatility      0.056616
Sharpe ratio          -0.346687
Calmar ratio          -0.265338
Stability              0.051921
Max drawdown          -0.078820
Omega ratio            0.941816
Sortino ratio         -0.458334
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.940856
Daily value at risk   -0.007211
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [39]:
%matplotlib inline
backtest_plot_tdx(df_account_value, 
             baseline_ticker = '603636', 
             baseline_start = '2021-01-01',
             baseline_end = '2021-12-16', value_col_name = 'total_assets')

Start date,2021-01-05
End date,2021-12-17
Total months,11
,Backtest
Annual return,-2.1%
Cumulative returns,-1.9%
Annual volatility,5.7%
Sharpe ratio,-0.35
Calmar ratio,-0.27
Stability,0.05
Max drawdown,-7.9%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'

In [ ]:
# from finrl.plot import get_daily_return, get_baseline_tdx
# from copy import deepcopy

# import pyfolio
# from pyfolio import timeseries

In [83]:
# def get_daily_return(df, value_col_name="account_value"):
#     df = deepcopy(df)
#     df["daily_return"] = df[value_col_name].pct_change(1)
#     df["date"] = pd.to_datetime(df["date"])
#     df.set_index("date", inplace=True, drop=True)
# #     df.index = df.index.tz_localize("UTC")
#     return pd.Series(df["daily_return"], index=df.index)

In [91]:
# def backtest_plot_tdx(
#     account_value,
#     baseline_start=config.START_TRADE_DATE,
#     baseline_end=config.END_DATE,
#     baseline_ticker="^DJI",
#     value_col_name="account_value",
# ):

#     df = deepcopy(account_value)
#     df["date"] = pd.to_datetime(df["date"])
    
#     test_returns = get_daily_return(df, value_col_name=value_col_name)

#     baseline_df = get_baseline_tdx(
#         ticker=baseline_ticker, start=baseline_start, end=baseline_end
#     )

#     baseline_df["date"] = baseline_df.index # pd.to_datetime(baseline_df["date"], format="%Y-%m-%d")
#     baseline_df.index = range(baseline_df.shape[0])
    
#     baseline_df = pd.merge(df[["date"]], baseline_df, how="left", on="date")
#     baseline_df = baseline_df.fillna(method="ffill").fillna(method="bfill")
#     baseline_returns = get_daily_return(baseline_df, value_col_name="close")
    
# #     print(test_returns.fillna(0))
# #     print('######################')
# #     print(baseline_returns.fillna(0))
# #     print('-----------------------------')
#     with pyfolio.plotting.plotting_context(font_scale=1.1):
#         pyfolio.create_full_tear_sheet(
#             returns=test_returns, benchmark_rets=baseline_returns, set_context=False
#         )